# IT Infrastructure Ontology - Analysis and Visualization

This notebook provides interactive analysis and visualization of the IT Infrastructure and Application Dependency Ontology.

## Features

- Load and query RDF/Turtle data
- Execute SPARQL queries
- Visualize infrastructure components
- Interactive network graphs
- Statistical analysis
- Export results

## Setup

First, install required packages:

```bash
pip install rdflib pandas matplotlib networkx pyvis plotly
```

In [ ]:
# Import required libraries
from rdflib import Graph, Namespace, RDF, RDFS
from rdflib.plugins.sparql import prepareQuery
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ Libraries imported successfully")

In [ ]:
# Define ontology namespaces
ONT = Namespace("http://example.org/it-infrastructure-ontology#")
INST = Namespace("http://example.org/instances#")

print("✓ Namespaces defined")
print(f"  Ontology: {ONT}")
print(f"  Instances: {INST}")

## Load Sample Data

Load the complex hybrid architecture sample data to analyze the infrastructure.

In [ ]:
# Load the ontology and sample data
g = Graph()

# Load ontology
ontology_path = '../ontology/it-infrastructure-ontology.ttl'
sample_data_path = '../ontology/sample-data-complex-hybrid.ttl'

try:
    g.parse(ontology_path, format='turtle')
    print(f"✓ Loaded ontology from {ontology_path}")
except Exception as e:
    print(f"✗ Failed to load ontology: {e}")

try:
    g.parse(sample_data_path, format='turtle')
    print(f"✓ Loaded sample data from {sample_data_path}")
except Exception as e:
    print(f"✗ Failed to load sample data: {e}")

print(f"\nTotal triples loaded: {len(g)}")

## Extract Physical Layer Nodes for Order Processing

Find all physical infrastructure components that support the order processing application through the dependency chain.

In [ ]:
# SPARQL query to find all physical layer nodes supporting order processing
query = """
PREFIX : <http://example.org/it-infrastructure-ontology#>
PREFIX inst: <http://example.org/instances#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?physical_node ?node_type ?name ?location ?status
WHERE {
  # Start with Order Processing business process
  inst:OrderProcessing :realized_by ?app .
  
  # Find all infrastructure components through dependency chains
  ?app (:uses|:deployed_as|:runs_on|:hosted_on|:communicates_via|:stored_on|:allocated_from)* ?physical_node .
  
  # Filter for physical layer components
  ?physical_node rdf:type ?node_type .
  FILTER(?node_type IN (
    :PhysicalServer, :VirtualMachine, :CloudInstance, :CloudService,
    :StorageArray, :StorageVolume, :Hypervisor
  ))
  
  # Get properties
  ?physical_node :name ?name .
  OPTIONAL { ?physical_node :location ?location }
  OPTIONAL { ?physical_node :lifecycle_status ?status }
}
ORDER BY ?node_type ?name
"""

# Execute query
results = g.query(query)

# Convert to DataFrame for better display
data = []
for row in results:
    data.append({
        'Physical Node': str(row.physical_node).split('#')[-1],
        'Type': str(row.node_type).split('#')[-1],
        'Name': str(row.name),
        'Location': str(row.location) if row.location else 'N/A',
        'Status': str(row.status) if row.status else 'N/A'
    })

df_physical = pd.DataFrame(data)
print(f"Found {len(df_physical)} physical layer nodes supporting Order Processing:")
print("" + "=" * 80)
display(df_physical)

In [ ]:
# Group by type for summary
type_summary = df_physical['Type'].value_counts()
print("\nPhysical Infrastructure Summary by Type:")
print("=" * 45)
for node_type, count in type_summary.items():
    print(f"{node_type:20}: {count:2d} nodes")

# Create visualization
plt.figure(figsize=(10, 6))
type_summary.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Physical Infrastructure Supporting Order Processing', fontsize=14, fontweight='bold')
plt.xlabel('Infrastructure Type', fontsize=12)
plt.ylabel('Number of Nodes', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Interactive Node Explorer

Interactive scatter plot where you can hover over nodes to see their detailed attributes.

In [ ]:
# Get detailed attributes for each node
detailed_query = """
PREFIX : <http://example.org/it-infrastructure-ontology#>
PREFIX inst: <http://example.org/instances#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?node ?name ?type ?prop ?value
WHERE {
  inst:OrderProcessing :realized_by ?app .
  ?app (:uses|:deployed_as|:runs_on|:hosted_on|:communicates_via|:stored_on|:allocated_from)* ?node .
  ?node rdf:type ?type .
  FILTER(?type IN (:PhysicalServer, :VirtualMachine, :CloudInstance, :CloudService, :StorageArray, :StorageVolume, :Hypervisor))
  ?node :name ?name .
  ?node ?prop ?value .
  FILTER(?prop != rdf:type && ?prop != :name)
}
"""

# Execute detailed query
detailed_results = g.query(detailed_query)

# Build node attributes dictionary
node_attrs = {}
for row in detailed_results:
    node_id = str(row.node).split('#')[-1]
    if node_id not in node_attrs:
        node_attrs[node_id] = {
            'name': str(row.name),
            'type': str(row.type).split('#')[-1],
            'attributes': {}
        }
    prop_name = str(row.prop).split('#')[-1]
    node_attrs[node_id]['attributes'][prop_name] = str(row.value)

print(f"Collected attributes for {len(node_attrs)} nodes")

In [ ]:
# Create interactive scatter plot
import plotly.graph_objects as go

# Prepare data for plotting
x_vals, y_vals, colors, hover_texts, node_ids = [], [], [], [], []
type_colors = {'PhysicalServer': 'red', 'VirtualMachine': 'blue', 'CloudInstance': 'green', 
               'CloudService': 'orange', 'StorageArray': 'purple', 'StorageVolume': 'brown', 'Hypervisor': 'pink'}

for i, (node_id, attrs) in enumerate(node_attrs.items()):
    x_vals.append(i % 10)
    y_vals.append(i // 10)
    colors.append(type_colors.get(attrs['type'], 'gray'))
    
    hover_info = f"<b>{attrs['name']}</b><br>Type: {attrs['type']}<br>"
    for key, value in list(attrs['attributes'].items())[:5]:
        hover_info += f"{key}: {value}<br>"
    hover_texts.append(hover_info)
    node_ids.append(node_id)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_vals, y=y_vals, mode='markers',
    marker=dict(size=15, color=colors, line=dict(width=2, color='black')),
    text=[attrs['name'] for attrs in node_attrs.values()],
    hovertemplate='%{hovertext}<extra></extra>',
    hovertext=hover_texts, customdata=node_ids
))

fig.update_layout(
    title='Interactive Physical Infrastructure Nodes<br><sub>Hover over nodes to see details</sub>',
    xaxis_title='Grid X', yaxis_title='Grid Y', showlegend=False, height=600
)
fig.show()

In [ ]:
# Node details function
def show_node_details(node_id):
    if node_id in node_attrs:
        attrs = node_attrs[node_id]
        print(f"\n{'='*60}")
        print(f"NODE: {attrs['name']}")
        print(f"TYPE: {attrs['type']}")
        print(f"{'='*60}")
        for key, value in attrs['attributes'].items():
            print(f"{key:25}: {value}")
        print(f"{'='*60}")
    else:
        print(f"Node '{node_id}' not found")

# Show example
if node_attrs:
    example_node = list(node_attrs.keys())[0]
    print(f"Example - showing details for: {example_node}")
    show_node_details(example_node)
    print(f"\nAvailable nodes: {', '.join(list(node_attrs.keys())[:10])}...")
    print("\nTo see other nodes, call: show_node_details('NodeName')")

## Application Layer Dependencies

Interactive network graph showing dependencies between application components.

In [ ]:
# Query for application and infrastructure dependencies
app_query = """
PREFIX : <http://example.org/it-infrastructure-ontology#>
PREFIX inst: <http://example.org/instances#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?source ?target ?source_name ?target_name ?source_type ?target_type ?relation
WHERE {
  ?source ?relation ?target .
  ?source rdf:type ?source_type .
  ?target rdf:type ?target_type .
  FILTER(?source_type IN (:Application, :Service, :API, :Database, :MessageQueue, :CacheService, :VirtualMachine, :CloudInstance, :PhysicalServer))
  FILTER(?target_type IN (:Application, :Service, :API, :Database, :MessageQueue, :CacheService, :VirtualMachine, :CloudInstance, :PhysicalServer))
  FILTER(?relation IN (:uses, :calls, :contains, :deployed_on, :runs_on, :hosted_on))
  ?source :name ?source_name .
  ?target :name ?target_name .
}
"""

app_results = g.query(app_query)

# Build network data
nodes = {}
edges = []

for row in app_results:
    source_id = str(row.source).split('#')[-1]
    target_id = str(row.target).split('#')[-1]
    
    nodes[source_id] = {
        'name': str(row.source_name),
        'type': str(row.source_type).split('#')[-1]
    }
    nodes[target_id] = {
        'name': str(row.target_name),
        'type': str(row.target_type).split('#')[-1]
    }
    
    edges.append({
        'source': source_id,
        'target': target_id,
        'relation': str(row.relation).split('#')[-1]
    })

print(f"Found {len(nodes)} components and {len(edges)} dependencies")

In [ ]:
# Create network graph
import networkx as nx
import plotly.graph_objects as go

# Build NetworkX graph
G = nx.DiGraph()

for node_id, attrs in nodes.items():
    G.add_node(node_id, **attrs)

for edge in edges:
    G.add_edge(edge['source'], edge['target'], relation=edge['relation'])

# Layout
pos = nx.spring_layout(G, k=3, iterations=50)

# Prepare plot data with arrows
edge_x, edge_y, arrow_x, arrow_y = [], [], [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    
    # Arrow position (80% along the edge)
    ax = x0 + 0.8 * (x1 - x0)
    ay = y0 + 0.8 * (y1 - y0)
    arrow_x.append(ax)
    arrow_y.append(ay)

node_x, node_y, node_text, node_colors = [], [], [], []
app_colors = {'Application': 'red', 'Service': 'blue', 'API': 'green', 
              'Database': 'orange', 'MessageQueue': 'purple', 'CacheService': 'brown',
              'VirtualMachine': 'lightblue', 'CloudInstance': 'lightgreen', 'PhysicalServer': 'pink'}

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_info = G.nodes[node]
    node_text.append(f"{node_info['name']}<br>Type: {node_info['type']}")
    node_colors.append(app_colors.get(node_info['type'], 'gray'))

# Create figure
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode='lines',
                        line=dict(width=1, color='gray'),
                        hoverinfo='none', showlegend=False))

# Add arrows
fig.add_trace(go.Scatter(x=arrow_x, y=arrow_y, mode='markers',
                        marker=dict(size=8, color='black', symbol='triangle-right'),
                        hoverinfo='none', showlegend=False))

# Add nodes
fig.add_trace(go.Scatter(x=node_x, y=node_y, mode='markers+text',
                        marker=dict(size=20, color=node_colors, line=dict(width=2, color='black')),
                        text=[nodes[node]['name'] for node in G.nodes()],
                        textposition='middle center',
                        hovertext=node_text, hoverinfo='text',
                        showlegend=False))

fig.update_layout(
    title='Application Layer Dependencies<br><sub>Hover over nodes for details</sub>',
    showlegend=False, hovermode='closest',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    height=700
)

fig.show()

## Interactive D3 Dependency Graph

Dynamic D3.js visualization with drag, select, and attribute display capabilities.

In [ ]:
from IPython.display import HTML
import json

# Prepare data for D3
d3_nodes = []
d3_links = []

for node_id, attrs in nodes.items():
    d3_nodes.append({
        'id': node_id,
        'name': attrs['name'],
        'type': attrs['type'],
        'group': hash(attrs['type']) % 10
    })

for edge in edges:
    d3_links.append({
        'source': edge['source'],
        'target': edge['target'],
        'relation': edge['relation']
    })

# Create D3 HTML
html_content = f"""
<div id="graph-container" style="width: 100%; height: 600px;"></div>
<div id="details" style="margin-top: 20px; padding: 10px; border: 1px solid #ccc; background: #f9f9f9;">
    <h4>Selection Details</h4>
    <div id="node-details">Click on a node to see details</div>
</div>

<script src="https://d3js.org/d3.v7.min.js"></script>
<script>
const nodes = {json.dumps(d3_nodes)};
const links = {json.dumps(d3_links)};

const width = 800, height = 600;
const svg = d3.select("#graph-container").append("svg")
    .attr("width", width).attr("height", height);

const simulation = d3.forceSimulation(nodes)
    .force("link", d3.forceLink(links).id(d => d.id).distance(100))
    .force("charge", d3.forceManyBody().strength(-300))
    .force("center", d3.forceCenter(width / 2, height / 2));

// Add arrow markers
svg.append("defs").selectAll("marker")
    .data(["arrow"]).enter().append("marker")
    .attr("id", "arrow").attr("viewBox", "0 -5 10 10")
    .attr("refX", 20).attr("refY", 0)
    .attr("markerWidth", 6).attr("markerHeight", 6)
    .attr("orient", "auto")
    .append("path").attr("d", "M0,-5L10,0L0,5").attr("fill", "#999");

// Links
const link = svg.append("g").selectAll("line")
    .data(links).enter().append("line")
    .attr("stroke", "#999").attr("stroke-width", 2)
    .attr("marker-end", "url(#arrow)");

// Nodes
const node = svg.append("g").selectAll("circle")
    .data(nodes).enter().append("circle")
    .attr("r", 10).attr("fill", d => d3.schemeCategory10[d.group])
    .call(d3.drag().on("start", dragstarted).on("drag", dragged).on("end", dragended))
    .on("click", function(event, d) {{
        d3.select("#node-details").html(`
            <strong>${{d.name}}</strong><br>
            Type: ${{d.type}}<br>
            ID: ${{d.id}}
        `);
    }});

// Labels
const label = svg.append("g").selectAll("text")
    .data(nodes).enter().append("text")
    .text(d => d.name).attr("font-size", 10)
    .attr("dx", 12).attr("dy", 4);

simulation.on("tick", () => {{
    link.attr("x1", d => d.source.x).attr("y1", d => d.source.y)
        .attr("x2", d => d.target.x).attr("y2", d => d.target.y);
    node.attr("cx", d => d.x).attr("cy", d => d.y);
    label.attr("x", d => d.x).attr("y", d => d.y);
}});

function dragstarted(event, d) {{
    if (!event.active) simulation.alphaTarget(0.3).restart();
    d.fx = d.x; d.fy = d.y;
}}

function dragged(event, d) {{
    d.fx = event.x; d.fy = event.y;
}}

function dragended(event, d) {{
    if (!event.active) simulation.alphaTarget(0);
    d.fx = null; d.fy = null;
}}
</script>
"""

HTML(html_content)